In [1]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

     |████████████████████████████████| 6.3 MB 20.1 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=6a06b7018b86f5b1691701aa5c71fed7ecab0e71492d40a04868af0cdf4853e0
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
%cd /content/drive/Shareddrives/Data606_Energy/data/futures

/content/drive/Shareddrives/Data606_Energy/data/futures


In [4]:
futures_symbols = {'BZ=F':'Brent Crude Oil','NG=F':'Natural Gas','CL=F':'Light Sweet Crude Oil','HO=F':'Heating Oil'}

In [ ]:
def get_futures_summaries(data,column_addition_string):
  # first break down the dataset by year
  data['year'] = data.index.year
  data['month'] = data.index.month
  unique_years = list(range(2000,2022))#data['year'].unique()
  mean_movement = []
  std_movement = []
  mean_volume = []
  std_volume = []
  # mean_open_interest = []
  # std_open_interest = []
  dates = []
  for year in unique_years:
    for month in range(1,13):
      dates.append(datetime(year=year,month=month,day=1))
      data_year_month = data[(data['year']==year)&(data['month']==month)]
      # get the average price movement within a day
      mean_movement.append(((data_year_month['High']-data_year_month['Low'])/data_year_month['Open']).mean())
      std_movement.append(((data_year_month['High']-data_year_month['Low'])/data_year_month['Open']).std())
      mean_volume.append(data_year_month['Volume'].mean())
      std_volume.append(data_year_month['Volume'].std())
  return {'Date':dates,'Mean Movement'+column_addition_string:mean_movement,'Std Movement'+column_addition_string:std_movement,'Mean Volume'+column_addition_string:mean_volume,'Std Volume'+column_addition_string:std_volume}#, 'Mean Open Interest'+column_addition_string:mean_open_interest,'Std Open Interest'+column_addition_string:std_open_interest}

In [ ]:
symbol_summaries = []
symbols_daily_data = []
symbols_weekly_data = []
for symbol in futures_symbols:
  # get summaries at the daily level
  daily_data = yf.download(tickers=symbol,start = '2000-01-01',end='2021-01-01',interval = '1d')
  symbols_daily_data.append(daily_data)
  daily_summaries = pd.DataFrame(get_futures_summaries(daily_data.copy(),' Daily '+symbol))
  # get summaries at the weekly level
  weekly_data = yf.download(tickers=symbol,start = '2000-01-01',end='2021-01-01',interval = '1wk')
  symbols_weekly_data.append(weekly_data)
  weekly_summaries = pd.DataFrame(get_futures_summaries(weekly_data.copy(),' Weekly '+symbol))
  # # get the summaries at the monthly level
  # monthly_data = yf.download(tickers=symbol,start = '2000-01-01',end='2021-01-01',interval = '1mo')
  # monthly_summaries = pd.DataFrame(get_futures_summaries(monthly_data.copy(),' Monthly '+symbol))
  # combine it all into one dataframe
  all_time_frames = pd.merge(daily_summaries,weekly_summaries,on='Date',how='outer')
  #all_time_frames = pd.merge(all_time_frames,monthly_summaries,on='Year')
  symbol_summaries.append(all_time_frames)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
# now go through the dataframes and create one large dataframe
try:
  del grand_data_frame
except:
  ...
for symbol_summary in symbol_summaries:
  # see if the dataframe has been started
  try:
    grand_data_frame
    # if it exists, merge the current dataframe to it
    grand_data_frame = pd.merge(grand_data_frame, symbol_summary,on='Date',how='outer')
  except:
    # if it doesn't exist, create it!
    grand_data_frame = symbol_summary.copy()

In [ ]:
grand_data_frame.to_csv('futures_data.csv',index=False)

In [ ]:
df = symbols_daily_data[1]
fig = make_subplots(specs=[[{"secondary_y": True}]])
df = symbols_daily_data[0]
fig = fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                ),secondary_y=True)
fig.add_trace(go.Bar(x=df.index, y=df['Volume']),
               secondary_y=False)

fig.show()